<a href="https://colab.research.google.com/github/BrianZ60/GPT/blob/main/GPT_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt # web get

with open("input.txt", 'r', encoding="utf-8") as f:
  text = f.read()

--2025-08-08 05:57:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-08-08 05:57:29 (18.4 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [51]:
print(f"Length of data set (characters): {len(text)}")

Length of data set (characters): 1115394


In [52]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [53]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
vocab_size


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [54]:
# tokenizer
# our tokens will be characters
ctoi = {ch:i for i,ch in enumerate(chars)} # dictionary of char to int
itoc = {i:ch for i,ch in enumerate(chars)}

# def encode(s: str): # takes in a string, outputs a list of ints
#   return [ctoi[c] for c in s]

# def decode(l): # takes in a list of ints, outputs a string
#   return ''.join(itoc[i] for i in l)

encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join(itoc[i] for i in l)

In [55]:
print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [56]:
# tokenize the entire data:
# dataset
import torch
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [57]:
split = int(0.9*len(data))

train_data = data[:split]
val_data = data[split:]

In [58]:
BLOCK_SIZE = 8 # maximum context length
train_data[:BLOCK_SIZE+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [59]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

# the size of the input goes from 1 to the block size

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [60]:
torch.manual_seed(1337)
BATCH_SIZE = 4
BLOCK_SIZE = 8

# dataloader
def get_batch(split_type):
  data = train_data if split_type == "train" else val_data
  # generate batch of data of inputs x and targets y
  offsets = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
  x = torch.stack([data[offset:offset+BLOCK_SIZE] for offset in offsets])
  y = torch.stack([data[offset+1:offset+BLOCK_SIZE+1] for offset in offsets])

  return x, y

batch_x, batch_y = get_batch("train")
print("inputs: ")
print(batch_x)

print("targets: ")
print(batch_y)

print("\n32 training examples:")

for b in range(BATCH_SIZE): # batch dimension
  print(f"batch #{b+1}")
  for t in range(BLOCK_SIZE): # time dimension
    context = batch_x[b, :t+1]
    target = batch_y[b,t]
    print(f"when input is {context} the target: {target}")

inputs: 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

32 training examples:
batch #1
when input is tensor([24]) the target: 43
when input is tensor([24, 43]) the target: 58
when input is tensor([24, 43, 58]) the target: 5
when input is tensor([24, 43, 58,  5]) the target: 57
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target: 39
batch #2
when input is tensor([44]) the target: 53
when input is tensor([44, 53]) the target: 56
when input is tensor([44, 53, 56]) the target: 1
when input is tenso

In [61]:
import torch
import torch.nn as nn

# bigram model only looks at prev token
class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)

  def forward(self, x, targets=None):
    # x: (B, T)
    logits = self.embedding_table(x) # (B, T, C)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

Test generate:

In [62]:
torch.manual_seed(1337)

model = BigramLanguageModel(vocab_size)

new_idx = model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)
print(decode(new_idx[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [63]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001) # adam with weight decay to prevent overfitting

In [64]:
BATCH_SIZE = 32

for steps in range(100):
  x_batch, y_batch = get_batch("train")

  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

4.587916374206543


In [65]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


xiKi-RJ:CgqVuUa!U?qMH.uk!sCuMXvv!CJFfx;LgRyJknOEti.?I&-gPlLyulId?XlaInQ'q,lT$
3Q&sGlvHQ?mqSq-eON
x?SP fUAfCAuCX:bOlgiRQWN:Mphaw
tRLKuYXEaAXxrcq-gCUzeh3w!AcyaylgYWjmJM?Uzw:inaY,:C&OECW:vmGGJAn3onAuMgia!ms$Vb q-gCOcPcUhOnxJGUGSPJWT:.?ujmJFoiNL&A'DxY,prZ?qdT;hoo'dHooXXlxf'WkHK&u3Q?rqUi.kz;?Yx?C&u3Qbfzxlyh'Vl:zyxjKXgC?
lv'QKFiBeviNxO'm!Upm$srm&TqViqiBD3HBP!juEOpmZJyF$Fwfy!PlvWPFC
&WDdP!Ko,px
x
tREOE;AJ.BeXkylOVD3KHp$e?nD,.SFbWWI'ubcL!q-tU;aXmJ&uGXHxJXI&Z!gHRpajj;l.
pTErIBjx;JKIgoCnLGXrJSP!AU-AcbczR?


Mathematical trick in self-attention

In [66]:
# toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2
x=torch.randn(B,T,C)

# tokens need to communicate but can only communicate with past tokens

In [67]:
# easiest way to do this is by averaging all previous contexts (weighted aggregation)

# version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    x_prev = x[b, :t+1] #(t+1, C)
    xbow[b,t] = torch.mean(x_prev, dim=0) # (C,)

In [68]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [69]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

We can do this much faster with matrix multi:

In [70]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # triangulates the matrix
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [71]:
weights = torch.tril(torch.ones(T, T))
weights /= weights.sum(1, keepdim=True)
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [72]:
# version 2
xbow2 = weights @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [73]:
# version 3 (what we're going to use)
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T)) # think of this as the relationships (affinities) between every token (init to 0 ofc)
wei = wei.masked_fill(tril == 0, float("-inf")) # we prevent tokens from interacting with future tokens
wei = torch.softmax(wei, dim=1)

xbow3 = wei @ x
xbow3[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

Works b/c softmax is $\frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}$

Cleanup and position embeddings

In [75]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
BATCH_SIZE = 32
BLOCK_SIZE = 8
MAX_ITERS = 3000
EVAL_INTERVAL = 300
LEARNING_RATE = 0.01
device = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 200
NUM_EMBD = 32 # number of embed dimensions

torch.manual_seed(1337)

# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt # web get

with open("input.txt", 'r', encoding="utf-8") as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

ctoi = {ch:i for i,ch in enumerate(chars)} # dictionary of char to int
itoc = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join(itoc[i] for i in l)

data = torch.tensor(encode(text))
split = int(0.9*len(data))
train_data = data[:split]
val_data = data[split:]

def get_batch(split_type):
  data = train_data if split_type == "train" else val_data
  # generate batch of data of inputs x and targets y
  offsets = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
  x = torch.stack([data[offset:offset+BLOCK_SIZE] for offset in offsets])
  y = torch.stack([data[offset+1:offset+BLOCK_SIZE+1] for offset in offsets])

  return x, y

def estimate_loss():
    out = {} # empty dict
    model.eval()

    with torch.inference_mode():
      for split in ["train", "val"]:
          losses = torch.zeros(EVAL_ITERS)
          for i in range(EVAL_ITERS):
              X, Y = get_batch(split)
              logits, loss = model(X, Y)
              losses[i] = loss.item()
          out[split] = losses.mean()

    return out

# bigram model only looks at prev token
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=NUM_EMBD)
    self.position_embedding_table = nn.Embedding(num_embeddings=BLOCK_SIZE, embedding_dim=NUM_EMBD)
    self.lm_head = nn.Linear(in_features=NUM_EMBD, out_features=vocab_size) # language modeling head

  def forward(self, idx, targets=None):
    # idx: (B, T)
    B, T = idx.shape
    token_emb = self.token_embedding_table(idx) # (B, T, C) # C here is num_embd
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = token_emb + pos_emb # (B, T, C)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


model = BigramLanguageModel()
model = model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

  x_batch, y_batch = get_batch("train")

  model.train()

  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter % EVAL_INTERVAL == 0:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.3176, val loss 4.3225
step 300: train loss 2.5404, val loss 2.5585
step 600: train loss 2.5178, val loss 2.5344
step 900: train loss 2.4965, val loss 2.5146
step 1200: train loss 2.5086, val loss 2.5240
step 1500: train loss 2.4873, val loss 2.5108
step 1800: train loss 2.4974, val loss 2.5201
step 2100: train loss 2.4958, val loss 2.5096
step 2400: train loss 2.4940, val loss 2.5098
step 2700: train loss 2.5050, val loss 2.5139


In [76]:
# v4: self-attention

torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# a single head performing self-attention

# query vector: what the token is looking for
# key vector: what the token contains (labels)
# wei becomes every query @ every key

HEAD_SIZE = 16
key = nn.Linear(C, HEAD_SIZE, bias=False) # key matrix
query = nn.Linear(C, HEAD_SIZE, bias=False) # query matrix
value = nn.Linear(C, HEAD_SIZE, bias=False) # value is the actual information tied to each key

k = key(x) # (B, T, HEAD_SIZE)
q = query(x) # (B, T, HEAD_SIZE)
# each token now has a key and query

# transpose k by swapping T and HEAD_SIZE dims
# wei = q @ k.transpose(1, -1) # (B, T, T)
wei = q @ k.transpose(1, -1) * HEAD_SIZE ** -0.5 # scaled attention

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = torch.softmax(wei, dim=2)

v = value(x)

# out = wei @ x
out = wei @ v #  (B, T, T) @ (B, T, HEAD_SIZE) --> (B, T, HEAD_SIZE)

wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)

In [77]:
class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(NUM_EMBD, head_size, bias=False)
    self.query = nn.Linear(NUM_EMBD, head_size, bias=False)
    self.value = nn.Linear(NUM_EMBD, head_size, bias=False)
    self.register_buffer("tril", torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE))) # we don't want it to be a parameter

  def forward(self, x):
    B,T,C = x.shape
    k=self.key(x)
    q=self.query(x)

    wei = q @ k.transpose(1, -1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf")) # slice out a TxT chunk of tril
    wei = torch.softmax(wei, dim=-1)

    v = self.value(x)
    return wei @ v

In [118]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
BATCH_SIZE = 32
BLOCK_SIZE = 8
MAX_ITERS = 5000
EVAL_INTERVAL = 500
LEARNING_RATE = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 200
NUM_EMBD = 32 # number of embed dimensions

torch.manual_seed(1337)

# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt # web get

with open("input.txt", 'r', encoding="utf-8") as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

ctoi = {ch:i for i,ch in enumerate(chars)} # dictionary of char to int
itoc = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join(itoc[i] for i in l)

data = torch.tensor(encode(text))
split = int(0.9*len(data))
train_data = data[:split]
val_data = data[split:]

def get_batch(split_type):
  data = train_data if split_type == "train" else val_data
  # generate batch of data of inputs x and targets y
  offsets = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
  x = torch.stack([data[offset:offset+BLOCK_SIZE] for offset in offsets])
  y = torch.stack([data[offset+1:offset+BLOCK_SIZE+1] for offset in offsets])
  x, y = x.to(device), y.to(device)

  return x, y

def estimate_loss():
    out = {} # empty dict
    model.eval()

    with torch.inference_mode():
      for split in ["train", "val"]:
          losses = torch.zeros(EVAL_ITERS)
          for i in range(EVAL_ITERS):
              X, Y = get_batch(split)
              logits, loss = model(X, Y)
              losses[i] = loss.item()
          out[split] = losses.mean()

    return out

# bigram model only looks at prev token
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=NUM_EMBD)
    self.position_embedding_table = nn.Embedding(num_embeddings=BLOCK_SIZE, embedding_dim=NUM_EMBD)
    self.sa_head = Head(NUM_EMBD)
    self.lm_head = nn.Linear(in_features=NUM_EMBD, out_features=vocab_size) # language modeling head

  def forward(self, idx, targets=None):
    # idx: (B, T)
    B, T = idx.shape
    token_emb = self.token_embedding_table(idx) # (B, T, C) # C here is num_embd
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = token_emb + pos_emb # (B, T, C)
    x = self.sa_head(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_crop = idx[:, -BLOCK_SIZE:] # only get last block_size tokens that way don't go out of scope for position embedding
      logits, _ = self(idx_crop) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


model = BigramLanguageModel()
model = model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

  x_batch, y_batch = get_batch("train")

  model.train()

  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter % EVAL_INTERVAL == 0:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.1854, val loss 4.1905
step 500: train loss 2.7224, val loss 2.7363
step 1000: train loss 2.5658, val loss 2.5730
step 1500: train loss 2.5229, val loss 2.5194
step 2000: train loss 2.4769, val loss 2.4866
step 2500: train loss 2.4586, val loss 2.4678
step 3000: train loss 2.4492, val loss 2.4562
step 3500: train loss 2.4305, val loss 2.4546
step 4000: train loss 2.4278, val loss 2.4358
step 4500: train loss 2.4180, val loss 2.4321

IUC-ELUIICIShaen, hwyar sanst es--
tpthe br'he p VO:
bay tUEVO:'dwo othaly Walod bere dherreuts.ogek,l betrath.

R KOENGHACTer'dr Heer, utttho ot suins, ivyRCe es'ust
y'Thitiy Ofaro:
o loumy wm.

Th ke ine REde er'spethe anetrses he ties ius cheh feltatous the.

A
Y  thatihneua nd thatrl cu mussee y wholuls:alde pgilwe h mis bule lr.
Cawiene se hallyo misin casut
er sten batretnl
St-Nhir,ve abdeSam! ste msou tohe'f shagcw uures ds baghine. Git.
TOas sh I'thmy, skss, h's lfant, r'ingthis nta;
The


Muli-Head Attention:

In [91]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

  def forward(self, x):
    return torch.cat([head(x) for head in self.heads], dim=-1) # concat over the C dimension

In [92]:
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=NUM_EMBD)
    self.position_embedding_table = nn.Embedding(num_embeddings=BLOCK_SIZE, embedding_dim=NUM_EMBD)
    self.sa_heads = MultiHeadAttention(4, NUM_EMBD//4) # split num_embd over num_heads
    self.lm_head = nn.Linear(in_features=NUM_EMBD, out_features=vocab_size) # language modeling head

  def forward(self, idx, targets=None):
    # idx: (B, T)
    B, T = idx.shape
    token_emb = self.token_embedding_table(idx) # (B, T, C) # C here is num_embd
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = token_emb + pos_emb # (B, T, C)
    x = self.sa_heads(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_crop = idx[:, -BLOCK_SIZE:] # only get last block_size tokens that way don't go out of scope for position embedding
      logits, _ = self(idx_crop) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

In [94]:
model = BigramLanguageModel()
model = model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

  x_batch, y_batch = get_batch("train")

  model.train()

  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter % EVAL_INTERVAL == 0:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.2276, val loss 4.2346
step 500: train loss 2.6234, val loss 2.6257
step 1000: train loss 2.4615, val loss 2.4663
step 1500: train loss 2.3995, val loss 2.4074
step 2000: train loss 2.3611, val loss 2.3825
step 2500: train loss 2.3289, val loss 2.3602
step 3000: train loss 2.3122, val loss 2.3338
step 3500: train loss 2.2996, val loss 2.3111
step 4000: train loss 2.2824, val loss 2.3140
step 4500: train loss 2.2576, val loss 2.2852

I wroumst.
KLO:
GAUSondt--'l, can, EA:
MUMET:
Cof cing.


O cows blin mess cise moks lis noonowe mand pemnenkar Coond eat.

's-
Or:
Thim of, rimenng ry. fow.
Thrithie Vodin deoor, veout thyoulds in risteng aive youn be plavickas tampe he hes tot ay sllir I'CAnoulst mad it-zis ant's ant samel velin:
The mo amy I tooritheth and furse oupporknot have ow sith thoull me id, my goou and be
As sest gay, Nyou thev:
Wobe cawte ind's furend; dy,
Shild And
Noldeanty solm. 'd Hoouen Lye.

Bun you songcouy f


Implement the FFN:

In [96]:
class FeedForward(nn.Module):
  # allows the tokens to think on all the data they gathered from sa individually
  def __init__(self, num_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(num_embd, num_embd),
        nn.ReLU()
    )

  def forward(self, x):
    return self.net(x)

In [ ]:
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=NUM_EMBD)
    self.position_embedding_table = nn.Embedding(num_embeddings=BLOCK_SIZE, embedding_dim=NUM_EMBD)
    self.sa_heads = MultiHeadAttention(4, NUM_EMBD//4) # split num_embd over num_heads
    self.ffn = FeedForward(NUM_EMBD)
    self.lm_head = nn.Linear(in_features=NUM_EMBD, out_features=vocab_size) # language modeling head

  def forward(self, idx, targets=None):
    # idx: (B, T)
    B, T = idx.shape
    token_emb = self.token_embedding_table(idx) # (B, T, C) # C here is num_embd
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = token_emb + pos_emb # (B, T, C)
    x = self.sa_heads(x)
    x = self.ffn(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_crop = idx[:, -BLOCK_SIZE:] # only get last block_size tokens that way don't go out of scope for position embedding
      logits, _ = self(idx_crop) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

Transformer Blocks

In [97]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffn = FeedForward(n_embd)

  def forward(self, x):
    return self.ffn(self.sa(x))

In [98]:
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=NUM_EMBD)
    self.position_embedding_table = nn.Embedding(num_embeddings=BLOCK_SIZE, embedding_dim=NUM_EMBD)
    self.blocks = nn.Sequential(
        Block(n_embd=NUM_EMBD, n_head=4),
        Block(n_embd=NUM_EMBD, n_head=4),
        Block(n_embd=NUM_EMBD, n_head=4),
    )
    self.lm_head = nn.Linear(in_features=NUM_EMBD, out_features=vocab_size) # language modeling head

  def forward(self, idx, targets=None):
    # idx: (B, T)
    B, T = idx.shape
    token_emb = self.token_embedding_table(idx) # (B, T, C) # C here is num_embd
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = token_emb + pos_emb # (B, T, C)
    x = self.blocks(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_crop = idx[:, -BLOCK_SIZE:] # only get last block_size tokens that way don't go out of scope for position embedding
      logits, _ = self(idx_crop) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

Residual Connections
TODO: explain it well pls

In [103]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffn = FeedForward(n_embd)

  def forward(self, x):
    x = x + self.sa(x)
    x = x + self.ffn(x)
    return x

Also need the projection layer in multi-head & ffn

In [100]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(NUM_EMBD, NUM_EMBD) # output projection


  def forward(self, x):
    out = torch.cat([head(x) for head in self.heads], dim=-1) # concat over the C dimension
    out = self.proj(out)
    return out

In [101]:
class FeedForward(nn.Module):
  # allows the tokens to think on all the data they gathered from sa individually
  def __init__(self, num_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(num_embd, 4*num_embd),
        nn.ReLU(),
        nn.Linear(4*num_embd, num_embd)
    )

  def forward(self, x):
    return self.net(x)

In [104]:
model = BigramLanguageModel()
model = model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

  x_batch, y_batch = get_batch("train")

  model.train()

  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1::
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.4411, val loss 4.4346
step 500: train loss 2.4106, val loss 2.4146
step 1000: train loss 2.2832, val loss 2.2907
step 1500: train loss 2.2114, val loss 2.2307
step 2000: train loss 2.1528, val loss 2.1880
step 2500: train loss 2.1156, val loss 2.1749
step 3000: train loss 2.0745, val loss 2.1333
step 3500: train loss 2.0570, val loss 2.1348
step 4000: train loss 2.0295, val loss 2.0918
step 4500: train loss 2.0189, val loss 2.0986

Wast;
Yorch
wer driangmore, for hall pere towslerenpery her.
The stand, I shoriurunedmain,
It uspikes
dand plity, spent that' pratith?

LUCHYVOLUED:
That unatt:
Cladnapen.
Ke forsiur,
Andso say your lume un traptaintorman reses as he tight
And will no-bereasellught sondst he vant'll and's thee rigalether- to dAk to cleangentlesigre.
Whonesser:
Hatughing brecinrefore when shonst.
To
Farl Lordeler.

CAmmon melo
mee wered as,
You a grooin.

BROMBEONE:
Bonid:
I wathost
Thanst I
To mulaw torsious ang


Layer normalization

come back to write implementation

In [105]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffn = FeedForward(n_embd)
    self.layer_norm_1 = nn.LayerNorm(n_embd)
    self.layer_norm_2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.layer_norm_1(x))
    x = x + self.ffn(self.layer_norm_2(x))
    return x

In [106]:
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=NUM_EMBD)
    self.position_embedding_table = nn.Embedding(num_embeddings=BLOCK_SIZE, embedding_dim=NUM_EMBD)
    self.blocks = nn.Sequential(
        Block(n_embd=NUM_EMBD, n_head=4),
        Block(n_embd=NUM_EMBD, n_head=4),
        Block(n_embd=NUM_EMBD, n_head=4),
        nn.LayerNorm(NUM_EMBD)
    )
    self.lm_head = nn.Linear(in_features=NUM_EMBD, out_features=vocab_size) # language modeling head

  def forward(self, idx, targets=None):
    # idx: (B, T)
    B, T = idx.shape
    token_emb = self.token_embedding_table(idx) # (B, T, C) # C here is num_embd
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = token_emb + pos_emb # (B, T, C)
    x = self.blocks(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_crop = idx[:, -BLOCK_SIZE:] # only get last block_size tokens that way don't go out of scope for position embedding
      logits, _ = self(idx_crop) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

Scaling it up

In [112]:
NUM_HEADS = 6
NUM_LAYERS = 6

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # lookup table where each row corresponds to each token
    # and the row values are the logits for the next token
    self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=NUM_EMBD)
    self.position_embedding_table = nn.Embedding(num_embeddings=BLOCK_SIZE, embedding_dim=NUM_EMBD)
    self.blocks = nn.Sequential(*[Block(n_embd=NUM_EMBD, n_head=NUM_HEADS) for _ in range(NUM_LAYERS)]) # * just unpacks the list
    self.layer_norm = nn.LayerNorm(NUM_EMBD)
    self.lm_head = nn.Linear(in_features=NUM_EMBD, out_features=vocab_size) # language modeling head

  def forward(self, idx, targets=None):
    # idx: (B, T)
    B, T = idx.shape
    token_emb = self.token_embedding_table(idx) # (B, T, C) # C here is num_embd
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = token_emb + pos_emb # (B, T, C)
    x = self.blocks(x)
    x = self.layer_norm(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_crop = idx[:, -BLOCK_SIZE:] # only get last block_size tokens that way don't go out of scope for position embedding
      logits, _ = self(idx_crop) # (B, T, C)
      logits = logits[:, -1, :] # (B, C) we only want the token probabilities for the last token
      probs = torch.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) Draws one sample using probabilities of probs
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

Dropouts

In [111]:
DROPOUT_PROB = 0.2

class FeedForward(nn.Module):
  # allows the tokens to think on all the data they gathered from sa individually
  def __init__(self, num_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(num_embd, 4*num_embd),
        nn.ReLU(),
        nn.Linear(4*num_embd, num_embd),
        nn.Dropout(DROPOUT_PROB)
    )

  def forward(self, x):
    return self.net(x)

In [109]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(NUM_EMBD, NUM_EMBD) # output projection
    self.dropout = nn.Dropout(DROPOUT_PROB)


  def forward(self, x):
    out = torch.cat([head(x) for head in self.heads], dim=-1) # concat over the C dimension
    out = self.dropout(self.proj(out))
    return out

In [110]:
class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(NUM_EMBD, head_size, bias=False)
    self.query = nn.Linear(NUM_EMBD, head_size, bias=False)
    self.value = nn.Linear(NUM_EMBD, head_size, bias=False)
    self.register_buffer("tril", torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE))) # we don't want it to be a parameter

    self.dropout = nn.Dropout(DROPOUT_PROB)

  def forward(self, x):
    B,T,C = x.shape
    k=self.key(x)
    q=self.query(x)

    wei = q @ k.transpose(1, -1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf")) # slice out a TxT chunk of tril
    wei = torch.softmax(wei, dim=-1)

    wei = self.dropout(wei)

    v = self.value(x)
    return wei @ v

In [117]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm.auto import tqdm

# hyperparameters
BATCH_SIZE = 64
BLOCK_SIZE = 64
MAX_ITERS = 5000
EVAL_INTERVAL = 500
LEARNING_RATE = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 200
NUM_EMBD = 96 # number of embed dimensions
NUM_HEADS = 6
NUM_LAYERS = 4
DROPOUT_PROB = 0.2

torch.manual_seed(1337)


model = BigramLanguageModel()
model = model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

for iter in tqdm(range(MAX_ITERS)):

  x_batch, y_batch = get_batch("train")

  model.train()

  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

  0%|          | 1/5000 [01:04<89:31:12, 64.47s/it]

step 0: train loss 4.2600, val loss 4.2527


  6%|▌         | 309/5000 [03:44<56:47,  1.38it/s]


KeyboardInterrupt: 